In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import mysql.connector
import numpy as np


db = mysql.connector.connect(host = 'localhost', 
                             username = 'root',
                             password = 'Ghanashyam&7500',
                             database = 'ecommerce')

cur = db.cursor()

# Calculate the moving average of order values for each customer over their order history.# 

In [7]:
query = """ select customer_id, order_purchase_timestamp, price,
avg(price) over(partition by customer_id order by order_purchase_timestamp 
rows between 2 preceding and current row) as mov_avg
from
(select orders.customer_id,
 orders.order_purchase_timestamp, payments.payment_value as price
from payments join orders
on payments.order_id = orders.order_id) as a;
"""


cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["customer_id","order_purchase_timestamp", "price", "mov_avg"])
df

,customer_id,order_purchase_timestamp,price,mov_avg
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
1,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
2,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
3,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
4,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
...,...,...,...,...
934969,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001
934970,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001
934971,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001
934972,ffffe8b65bbe3087b653a978c870db99,2017-09-29 14:07:03,18.37,18.370001


In [8]:
query = """ select years, months, sum(payments)
over(order by years, months) cumulative_sales 
from
(select year(orders.order_purchase_timestamp) as years,
month(orders.order_purchase_timestamp) as months,
round(sum(payments.payment_value),2) as payments from orders join payments
on orders.order_id = payments.order_id
group by years, months
order by years, months) as g;
"""

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["years","months", "cumulative_sales"])
df

,years,months,cumulative_sales
0,2016,9,2.270160e+03
1,2016,10,5.340845e+05
2,2016,12,5.342611e+05
3,2017,1,1.780653e+06
4,2017,2,4.407826e+06
5,2017,3,8.456598e+06
6,2017,4,1.221669e+07
7,2017,5,1.755296e+07
8,2017,6,2.215445e+07
9,2017,7,2.748589e+07


# Calculate the year-over-year growth rate of total sales.

In [7]:
query = """
with a as
(select year(orders.order_purchase_timestamp) as years,
round(sum(payments.payment_value),2) as payments from orders join payments
on orders.order_id = payments.order_id
group by years
order by years)
select years, ((payments - lag(payments, 1) over (order by years )) 
/ lag(payments, 1) over(order by years)) * 100 year_over_year_percentage_growth from a 
"""

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["years","yoy % growth"])
df

,years,yoy % growth
0,2016,NaN
1,2017,12112.703758
2,2018,20.000924


# Calculate the retention rate of customers, defined as the percentage of customers who make another purchase within 6 months of their first purchase

In [2]:
query = """ 
with a as
(select customers.customer_id, min(orders.order_purchase_timestamp) first_order
from customers join orders
on orders.customer_id = customers.customer_id 
group by customers.customer_id),

b as(select a.customer_id, count(distinct orders.order_purchase_Timestamp)  next_order 
from a join orders
on orders.customer_id = a.customer_id
and orders.order_purchase_timestamp > first_order
and orders.order_purchase_timestamp < date_add(first_order, interval 6 month)
group by a.customer_id)

select 100 *(count(distinct a.customer_id)/ count(distinct b.customer_id))
from a left join b
on a.customer_id = b.customer_id;
"""

cur.execute(query)
data = cur.fetchall()
data

[(None,)]

# Identify the top 3 customers who spent the most money in each year.

In [4]:
query = """ 
select years, customer_id, d_rank, payment
from
(select year(orders.order_purchase_timestamp) years, orders.customer_id, sum(payments.payment_value) payment,
dense_rank() over ( partition by year(orders.order_purchase_timestamp)
order by sum(payments.payment_value)desc) d_rank
from  orders join payments
on payments.order_id = orders.order_id
group by year(orders.order_purchase_timestamp), orders.customer_id) as a
where d_rank <= 3;

"""

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["years", "id", "payment", "rank"])
df

,years,id,payment,rank
0,2016,a9dc96b027d1252bbac0a9b72d837fc6,1,12811.950439
1,2016,1d34ed25963d5aae4cf3d7f3a4cda173,2,12606.659912
2,2016,4a06381959b6670756de02e07b83815f,3,11050.020264
3,2017,1617b1357756262bfa56ab541c47bc16,1,122976.720703
4,2017,c6e2731c5b391845f6800c97401a43a9,2,62363.790527
5,2017,3fd6777bbce08a352fddd04e4a7cc8f6,3,60539.941406
6,2018,ec5b2ba62e574342386871631fafd3fc,1,65473.918945
7,2018,f48d464a0baaea338cb25f816991ab1f,2,62299.889648
8,2018,e0a2412720e9ea4f26c1ac985f6a7358,3,43284.959473
